Build a regression model.

In [4]:
# Load Packages
#!pip install requests
import requests
import json
import re
#!pip install pandas
import pandas as pd
import os # use this to access your environment variables
#!pip install plotly
import plotly.express as px
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
#!pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt
#!pip install scipy
from scipy.stats import ttest_ind
import sqlite3
#!pip install scikit-learn
from sklearn.preprocessing import StandardScaler
#!pip install statsmodels
import statsmodels.api as sm

   ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
    --------------------------------------- 0.1/9.2 MB 3.6 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/9.2 MB 16.9 MB/s eta 0:00:01
   ---------- ----------------------------- 2.4/9.2 MB 19.3 MB/s eta 0:00:01
   --------------- ------------------------ 3.5/9.2 MB 20.2 MB/s eta 0:00:01
   ------------------- -------------------- 4.5/9.2 MB 19.3 MB/s eta 0:00:01
   ------------------------ --------------- 5.7/9.2 MB 20.2 MB/s eta 0:00:01
   ---------------------------- ----------- 6.6/9.2 MB 21.0 MB/s eta 0:00:01
   -------------------------------- ------- 7.5/9.2 MB 20.9 MB/s eta 0:00:01
   ------------------------------------- -- 8.6/9.2 MB 21.1 MB/s eta 0:00:01
   ---------------------------------------  9.2/9.2 MB 20.3 MB/s eta 0:00:01
   ---------------------------------------- 9.2/9.2 MB 19.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/302.2 kB ? eta -:--:--
   -----------

In [ ]:
# Model Building 
# Dependent Variable - Total Bikes in use
# Independent Variables -Avg_POI_Count, Yelp_Price_Category, Yelp_Avg_Rating, Yelp_Avg_Review_Count
complete_df = pd.read_csv(r"C:\Users\aksha\Documents\LHL\LHL Assignments\Week 5\Python Statistical Modeling Project\Python Project\Statistical_Modeling_Python\data\Merged_Data_17th.csv")

model_df = complete_df[['Total_Bikes_In_Use', 'Avg_POI_Count', 'Yelp_Price_Category', 'Yelp_Avg_Rating', 'Yelp_Avg_Review_Count']]

model_df.describe(include = 'all')

# Check for NAs
na_col_count = model_df.isna().sum(axis = 0)
na_col_count

model_df.info()

df_encoded = pd.get_dummies(model_df, columns=['Yelp_Price_Category'], drop_first=True)

# Check data types
print(df_encoded.dtypes)
# Check for missing values
print(df_encoded.isnull().sum())

print(df_encoded.shape)
print(model_df.shape)

# Add a constant term for the intercept
X = sm.add_constant(df_encoded[['Avg_POI_Count', 'Yelp_Price_Category_Expensive', 'Yelp_Avg_Rating', 'Yelp_Avg_Review_Count']])

# Fit the regression model
model = sm.OLS(model_df['Total_Bikes_In_Use'], X.astype(float)).fit()
model.summary()
# Values of R-squared and Adjusted R-squared are very low. None of the factors is significantly contributing to the variation Total Bikes in Use

In [ ]:
# Fit the model with Scaling 


df_encoded = pd.get_dummies(model_df, columns=['Yelp_Price_Category'], drop_first=True)

# Extract features and target variable
X = df_encoded[['Avg_POI_Count', 'Yelp_Price_Category_Expensive', 'Yelp_Avg_Rating', 'Yelp_Avg_Review_Count']]
y = model_df['Total_Bikes_In_Use']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Add a constant term for the intercept
X_scaled = sm.add_constant(X_scaled)

# Fit the regression model
model = sm.OLS(y, X_scaled).fit()

# Display the summary of the regression
print(model.summary())
# Still no change in R-squared and Adjusted R-squared. None of the factors is significantly contributing to the variation Total Bikes in Use

Provide model output and an interpretation of the results. 

In [ ]:
print(model.summary())

# The model, doesn't seem to provide a good fit to the data. 
# The coefficients are not statistically significant, and the R-squared values are low.
# Inclusion of additional parameters and more data might lead to a better performance.
# Even in correlation plot we can see there is no correlation of any of the quantitative fields with the dependent variable

# Calculate the correlation matrix
corr_matrix = model_df[['Total_Bikes_In_Use','Avg_POI_Count', 'Yelp_Avg_Rating', 'Yelp_Avg_Review_Count']].corr()

# Create a heatmap 
plt.figure(figsize=(6, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Heatmap')
plt.show()

# Stretch

How can you turn the regression model into a classification model?